In [1]:
from time import time
from GeneralisedWick import *
import CC
from pyscf import cc, mp
import pickle

In [2]:
with open("CCDEquations.pkl", 'rb') as f:
    up = pickle.Unpickler(f)
    d = up.load()
    energyEquation = d["energyEquation"]
    doublesAmplitudeEquation = d["doublesAmplitudeEquation"]
    fockTensor = d["fockTensor"]
    h2Tensor = d["h2Tensor"]
    t2Tensor = d["t2Tensor"]

In [3]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'Be 0 0 0'
#mol.basis = 'sto-3g'
#mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])
fockTensor.array = fock

fockTensor.assignDiagramArrays(vacuum)
h2Tensor.assignDiagramArrays(vacuum)
t2Tensor.getShape(vacuum)

In [4]:
trueMP2 = mp.MP2(mf)
trueMP2.kernel()

(-0.024311010805774455,
 (array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             2.16840434e-19],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  2.16840434e-19,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00, -2.16840434e-19,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00, -2.16840434

In [5]:
trueCCD = cc.CCSD(mf)
old_update_amps = trueCCD.update_amps
def update_amps(t1, t2, eris):
    t1, t2 = old_update_amps(t1, t2, eris)
#    print(t1)
    return (np.zeros_like(t1[0]), np.zeros_like(t1[1])), t2
trueCCD.update_amps = update_amps
print(trueCCD.kernel())

(-0.046469771227442426, (array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])), (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,

In [6]:
residualTensor = Tensor("R", ['p', 'p'], ['h', 'h'])

In [7]:
print(doublesAmplitudeEquation)

2.0 * v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -1.0 * v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -1.0 * v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 2.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 1.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -2.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -2.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 1.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -2.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 1.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 1.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -2.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{2}}
 + -1.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 2.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 2.0 * f_{p_{0}}^{p_{3}}{t_{2}}_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -1.0 * f_{p_{0}}^{p_{3}}{t_{2}}_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 2.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -1.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -1.

In [8]:
residualTensor.array = contractTensorSum(doublesAmplitudeEquation)

In [9]:
residualTensor.array

array([[[[ 0.00660172,  0.01319636],
         [ 0.01319636,  0.12607   ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.01871811, -0.00427657],
         [ 0.048972  ,  0.05697686]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00660172,  0.01319636],
         [ 0.01319636,  0.12607   ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [10]:
contractTensorSum(doublesAmplitudeEquation.collectIsomorphicTerms())

array([[[[ 0.00660172,  0.01319636],
         [ 0.01319636,  0.12607   ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.01871811,  0.01347295],
         [ 0.03122248,  0.05697686]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00660172,  0.01319636],
         [ 0.01319636,  0.12607   ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [11]:
residualTensor.array == contractTensorSum(doublesAmplitudeEquation.collectIsomorphicTerms())

array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[False, False],
         [False, False]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [False,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True, False],
         [False,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [False,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        

In [12]:
h2Tensor.diagrams[12].array

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [13]:
t2Tensor.array = np.zeros_like(t2Tensor.array)

In [14]:
contractTensorSum(energyEquation)

0.0

In [15]:
CC.iterateDoublesAmplitudes(t2Tensor, residualTensor, fock)

array([[[[-0.00068921, -0.00255098],
         [-0.00255098, -0.16426883]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00187911,  0.00076978],
         [-0.00881497, -0.04954694]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00068921, -0.00255098],
         [-0.00255098, -0.16426883]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [16]:
t2Tensor.array

array([[[[-0.00068921, -0.00255098],
         [-0.00255098, -0.16426883]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00187911,  0.00076978],
         [-0.00881497, -0.04954694]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00068921, -0.00255098],
         [-0.00255098, -0.16426883]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [17]:
residualTensor.array

array([[[[ 0.00660172,  0.01319636],
         [ 0.01319636,  0.12607   ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.01871811, -0.00427657],
         [ 0.048972  ,  0.05697686]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00660172,  0.01319636],
         [ 0.01319636,  0.12607   ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [18]:
t2Tensor.array = np.zeros_like(t2Tensor.array)
residualTensor.array = contractTensorSum(doublesAmplitudeEquation)
testMP2amps = CC.iterateDoublesAmplitudes(t2Tensor, residualTensor, fockTensor.array)

In [19]:
t2Tensor.array = (1./3.) * testMP2amps + (1./6.) * testMP2amps.swapaxes(0,1)
#t2Tensor.array = testMP2amps

In [20]:
print(energyEquation)

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}


In [21]:
contractTensorSum(energyEquation)

-0.024311010819762693

In [22]:
testMP2amps

array([[[[-0.00068921, -0.00255098],
         [-0.00255098, -0.16426883]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00187911,  0.00076978],
         [-0.00881497, -0.04954694]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00068921, -0.00255098],
         [-0.00255098, -0.16426883]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [23]:
trueMP2.kernel()[1][1].swapaxes(0,2).swapaxes(1,3) + trueMP2.kernel()[1][0].swapaxes(0,2).swapaxes(1,3)

array([[[[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00093956,  0.00038489],
         [-0.00440748, -0.02477347]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [24]:
truemp2amps = trueMP2.kernel()[1][1].swapaxes(0,2).swapaxes(1,3)

In [25]:
truemp2amps

array([[[[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.00093956, -0.00121257],
         [-0.00281002, -0.02477347]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]]],


       [[[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[

In [26]:
trueMP2.kernel()[1][1]

array([[[[-0.00034461, -0.        , -0.        , -0.        ,
          -0.00093956, -0.        , -0.        ],
         [-0.        , -0.00034461, -0.        , -0.        ,
          -0.        , -0.        , -0.00093956],
         [-0.        , -0.        , -0.00034461, -0.        ,
          -0.        , -0.00093956, -0.        ],
         [-0.        , -0.        , -0.        , -0.00200884,
          -0.        , -0.        , -0.        ],
         [-0.00093956, -0.        , -0.        , -0.        ,
          -0.00260934, -0.        , -0.        ],
         [-0.        , -0.        , -0.00093956, -0.        ,
          -0.        , -0.00260934, -0.        ],
         [-0.        , -0.00093956, -0.        , -0.        ,
          -0.        , -0.        , -0.00260934]],

        [[-0.00127549, -0.        , -0.        , -0.        ,
          -0.00121257, -0.        , -0.        ],
         [-0.        , -0.00127549, -0.        , -0.        ,
          -0.        , -0.        , -0.0

In [27]:
truemp2amps = trueMP2.init_amps()[1][0].swapaxes(0,2).swapaxes(1,3)

In [28]:
trueCCD.kernel()[2]

(array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        ,  0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ,  0.        ,  0.        ,
            0.00113838,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ,  0.        ,
            0.        

In [29]:
trueCCDamps = trueCCD.kernel()[2][1].swapaxes(0,2).swapaxes(1,3)

In [30]:
trueCCD.init_amps()

(-0.02431101081976273,
 (array([[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
          -6.24643615e-12, -0.00000000e+00, -0.00000000e+00,
          -0.00000000e+00],
         [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
          -1.14538097e-09, -0.00000000e+00, -0.00000000e+00,
          -0.00000000e+00]]),
  array([[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
          -6.24643615e-12, -0.00000000e+00, -0.00000000e+00,
          -0.00000000e+00],
         [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
          -1.14538097e-09, -0.00000000e+00, -0.00000000e+00,
          -0.00000000e+00]])),
 (array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  1.08420217e-19,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+0

In [31]:
trueCCD.kernel()[2][1]

array([[[[ 7.25036639e-05,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00, -8.32519695e-04,  0.00000000e+00,
           0.00000000e+00],
         [ 0.00000000e+00,  7.25036639e-05,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          -8.32519695e-04],
         [ 0.00000000e+00,  0.00000000e+00,  7.25036639e-05,
           0.00000000e+00,  0.00000000e+00, -8.32519695e-04,
           0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          -1.69925945e-03,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00],
         [-8.32519695e-04,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00, -2.56443004e-03,  0.00000000e+00,
           0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -8.32519695e-04,
           0.00000000e+00,  0.00000000e+00, -2.56443004e-03,
           0.00000000e+00],
         [ 0.00000000e+00, -8.32519695e-04,  0.00000000e+00,
           0.00000000e+00,  0.00000000e

In [32]:
print(energyEquation)

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}


In [33]:
print(doublesAmplitudeEquation)

2.0 * v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -1.0 * v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -1.0 * v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 2.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 1.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -2.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -2.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 1.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -2.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 1.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 1.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -2.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{2}}
 + -1.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 2.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 2.0 * f_{p_{0}}^{p_{3}}{t_{2}}_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -1.0 * f_{p_{0}}^{p_{3}}{t_{2}}_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 2.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -1.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -1.

In [34]:
len(doublesAmplitudeEquation.summandList)

708

In [35]:
t2Tensor.array = np.zeros_like(t2Tensor.array)

In [36]:
t2Tensor.array = truemp2amps

In [37]:
t2Tensor.array = trueCCDamps

In [38]:
residualTensor.array = np.zeros_like(residualTensor.array)

In [39]:
residualTensor.array = contractTensorSum(doublesAmplitudeEquation)

In [40]:
t2Tensor.array = CC.iterateDoublesAmplitudes(t2Tensor, residualTensor, fock)

In [41]:
contractTensorSum(energyEquation)

-0.04646975755538652

In [42]:
CC.convergeDoublesAmplitudes(t2Tensor, energyEquation, doublesAmplitudeEquation, fockTensor)

0.0
-0.024311010819762696
-0.03554981543184186
-0.041001581354297136
-0.04370736381045784
-0.04506823009333392
-0.04575705872273241
-0.046106868382493255
-0.04628483280767276
-0.0463754703841619
-0.046421665143156814
-0.04644522051463634
-0.046457235905851
-0.04646336638372021
-0.04646649483543366
-0.04646809153055461
-0.04646890652857674
-0.04646932255597538
-0.046469534934192674
-0.046469643355515095
-0.046469698707314856
-0.04646972696638695
-0.04646974139387722
-0.04646974875982572
-0.04646975252053947
-0.04646975444059872
-0.04646975542090332
-0.04646975592140895
-0.04646975617694844
-0.04646975630741762
-0.04646975637403055
-0.04646975640804077
-0.04646975642540521
-0.04646975643427092
-0.04646975643879742
-0.04646975644110851
-0.046469756442288464
-0.04646975644289093
-0.046469756443198514
-0.04646975644335556
-0.046469756443435754
-0.04646975644347669
-0.04646975644349759
-0.046469756443508245
-0.046469756443513706
[[[[ 7.25057996e-05 -1.22582488e-03]
   [-1.22582488e-03 -1.717

In [43]:
with open("CCSDEquations.pkl", 'rb') as f:
    up = pickle.Unpickler(f)
    d = up.load()
    energyEquation = d["energyEquation"]
    singlesAmplitudeEquation = d["singlesAmplitudeEquation"]
    doublesAmplitudeEquation = d["doublesAmplitudeEquation"]
    fockTensor = d["fockTensor"]
    h2Tensor = d["h2Tensor"]
    t1Tensor = d["t1Tensor"]
    t2Tensor = d["t2Tensor"]

In [44]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'Be 0 0 0'
#mol.basis = 'sto-3g'
#mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])
fockTensor.array = fock

fockTensor.assignDiagramArrays(vacuum)
h2Tensor.assignDiagramArrays(vacuum)
t1Tensor.getShape(vacuum)
t2Tensor.getShape(vacuum)

In [45]:
trueMP2 = mp.MP2(mf)
trueMP2.kernel()

(-0.024311010805774455,
 (array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             2.16840434e-19],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  2.16840434e-19,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00, -2.16840434e-19,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00, -2.16840434

In [46]:
trueCCSD = cc.CCSD(mf)
trueCCSD.kernel()

(-0.04675404036274192,
 (array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]]),
  array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,

In [47]:
CC.convergeCCSDAmplitudes(t1Tensor,t2Tensor, energyEquation, singlesAmplitudeEquation, doublesAmplitudeEquation, fockTensor)

0.0
-0.012155505409881352
-0.021120122833355213
-0.027753911251381977
-0.03264348917097595
-0.03626051593965905
-0.038934520189230395
-0.04091842330653703
-0.042391290448244315
-0.043488212444914257
-0.04430588378999258
-0.04491700940405635
-0.04537408115497582
-0.04571669161006606
-0.04597358798596588
-0.04616658711920865
-0.046311567860063754
-0.04642067490681796
-0.0465027404632912
-0.046564580730038066
-0.04661113342215027
-0.046646248993933284
-0.046672697107124024
-0.0466926642048522
-0.04670770664681517
-0.046719071402615456
-0.0467276334597916
-0.0467341067702189
-0.04673898289598233
-0.046742672158017376
-0.046745450181895805
-0.04674755371584631
-0.046749136808502754
-0.046750336652085195
-0.04675123893410157
-0.0467519235491277
-0.046752437837725466
-0.046752828595115384
-0.04675312172918861
-0.046753344833711644
-0.046753511899303254
-0.046753639325838724
-0.04675373452503767
-0.04675380733322926
-0.046753861566365726
-0.04675390318591779
-0.04675393406991665
-0.04675395787

KeyboardInterrupt: 

In [ ]:
len(doublesAmplitudeEquation.summandList)

6204

In [ ]:
len(doublesAmplitudeEquation.collectIsomorphicTerms().summandList)

271